In [1]:
# Use conda/analysis3-24.07 env
import netCDF4 as nc
import numpy as np

In [2]:
from pyproj import Transformer
from pyproj import CRS
ps_to_ll = Transformer.from_crs( "EPSG:3031","EPSG:4326", always_xy=True) 

In [3]:
# Open the NetCDF file
dataset = nc.Dataset('/g/data/tm70/ek4684/Bedmachine_data/BedMachineAntarctica_2020-07-15_v02.nc', 'r')
# Extract the latitude and longitude arrays
y = dataset.variables['y'][:]
x = dataset.variables['x'][:]
# Create 2D coordinate grids
X, Y = np.meshgrid(x, y)

# Flatten and transform
lon_flat, lat_flat = ps_to_ll.transform(X.ravel(), Y.ravel())
lon_flat = (lon_flat + 180) % 360 - 180  # Normalize longitudes

# Reshape to 2D
new_longitudes = lon_flat.reshape(X.shape)
new_latitudes = lat_flat.reshape(Y.shape)

In [5]:
# Define new file path
new_file_path = '/g/data/tm70/ek4684/Bedmachine_data/BedMachineAntarctica_2020-07-15_v02_pyproj.nc'
new_dataset = nc.Dataset(new_file_path, 'w', format='NETCDF4')

# Copy dimensions from the original dataset to the new file
for name, dimension in dataset.dimensions.items():
    new_dataset.createDimension(name, len(dimension) if not dimension.isunlimited() else None)

# Copy variables (excluding x and y) from the original dataset to the new file
for name, variable in dataset.variables.items():
    if name not in ['x', 'y']:  # Exclude 'x' and 'y', as we are replacing them with lat/lon
        new_var = new_dataset.createVariable(name, variable.datatype, variable.dimensions)
        new_var[:] = variable[:]
        # Copy all attributes except the fill value
        for attr_name in variable.ncattrs():
            if attr_name != '_FillValue':
                new_var.setncattr(attr_name, variable.getncattr(attr_name))

# Create new lat and lon variables with the correct dimensions
lat_var = new_dataset.createVariable('lat', 'f4', ('y', 'x'))  # Latitude (2D)
lon_var = new_dataset.createVariable('lon', 'f4', ('y', 'x'))  # Longitude (2D)

# Assign the computed latitudes and longitudes to the new variables
lat_var[:, :] = new_latitudes  # Ensure latitudes are assigned correctly as 2D
lon_var[:, :] = new_longitudes  # Ensure longitudes are assigned correctly as 2D

# Set attributes for the new variables
lat_var.units = "degrees_north"
lon_var.units = "degrees_east"

# Close the new dataset
new_dataset.close()

# Close the original dataset
dataset.close()

print(f"New file saved as: {new_file_path}")

New file saved as: /g/data/tm70/ek4684/Bedmachine_data/BedMachineAntarctica_2020-07-15_v02_pyproj.nc
